MFCC as feature

In [1]:
import pandas as pd
import numpy as np
import os
import wave
from scipy import signal
from scipy.io import wavfile
import IPython.display as ipd
import librosa
import librosa.display
from sklearn.preprocessing import LabelEncoder

In [2]:
data=pd.read_csv("archive/speakers_all.csv",index_col='speakerid')
data = data.iloc[:,:8]
data = data[data['file_missing?']==False]
data

,age,age_onset,birthplace,filename,native_language,sex,country,file_missing?
speakerid,,,,,,,,
1,27.0,9.0,"virginia, south africa",afrikaans1,afrikaans,female,south africa,False
2,40.0,5.0,"pretoria, south africa",afrikaans2,afrikaans,male,south africa,False
418,43.0,4.0,"pretoria, transvaal, south africa",afrikaans3,afrikaans,male,south africa,False
1159,26.0,8.0,"pretoria, south africa",afrikaans4,afrikaans,male,south africa,False
1432,19.0,6.0,"cape town, south africa",afrikaans5,afrikaans,male,south africa,False
...,...,...,...,...,...,...,...,...
766,46.0,5.0,"lagos, nigeria",yoruba3,yoruba,female,nigeria,False
851,46.0,12.0,"lagos, nigeria",yoruba4,yoruba,male,nigeria,False
2023,47.0,2.0,"ibadan, nigeria",yoruba5,yoruba,female,nigeria,False


In [3]:
INPUT_DIR = 'archive/recordings/recordings'
OUTPUT_DIR = 'results'
parent_list = os.listdir(INPUT_DIR)

print(set(data['filename']+'.mp3') - set(parent_list))

data = data.drop(data[data['filename']=='sinhalese1'].index)
data = data.drop(data[data['filename']=='nicaragua'].index)
data

{'nicaragua.mp3', 'sinhalese1.mp3'}


,age,age_onset,birthplace,filename,native_language,sex,country,file_missing?
speakerid,,,,,,,,
1,27.0,9.0,"virginia, south africa",afrikaans1,afrikaans,female,south africa,False
2,40.0,5.0,"pretoria, south africa",afrikaans2,afrikaans,male,south africa,False
418,43.0,4.0,"pretoria, transvaal, south africa",afrikaans3,afrikaans,male,south africa,False
1159,26.0,8.0,"pretoria, south africa",afrikaans4,afrikaans,male,south africa,False
1432,19.0,6.0,"cape town, south africa",afrikaans5,afrikaans,male,south africa,False
...,...,...,...,...,...,...,...,...
766,46.0,5.0,"lagos, nigeria",yoruba3,yoruba,female,nigeria,False
851,46.0,12.0,"lagos, nigeria",yoruba4,yoruba,male,nigeria,False
2023,47.0,2.0,"ibadan, nigeria",yoruba5,yoruba,female,nigeria,False


In [29]:
#features = feature_extractor(parent_list, feature_type='mfcc')

features = np.zeros(shape=(2138,10,862))
i=0
SAMPLE_RATE = 22050
dire = 'archive/recordings/recordings/'
for file in parent_list[:]:
    f_name = str(dire+file)
    y, sr = librosa.load(f_name, sr=SAMPLE_RATE, duration = 20 )     # duration 10s
    mfcc = librosa.feature.mfcc(y=y, sr=SAMPLE_RATE, n_mfcc =10)
    if len(mfcc[1]) < 862:
        offset = 862 - len(mfcc[1])
        mfcc= np.pad(mfcc,((0,0),(offset,0)), 'constant')
        
    features[i,:,:]=mfcc
    i+=1

D:\py\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [30]:
np.save("knn_features.npy",features)

In [31]:
X = np.load("knn_features.npy")
print(np.shape(X))

(2138, 10, 862)


In [32]:
y_accent = data['native_language']

encoder1 = LabelEncoder()
encoder1.fit(y_accent)
y_accent_ = encoder1.transform(y_accent)
y_accent_ = to_categorical(np.array(y_accent_),dtype='float32')
print(np.shape(y_accent_))  #200 categories.

(2138, 200)


In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_accent_train, y_accent_test = train_test_split(X, y_accent_, test_size=0.2, random_state=10)

mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std
X_train= np.expand_dims(X_train, axis=3)
X_test= np.expand_dims(X_test, axis=3)

#X_train, X_val, y_accent_train, y_accent_val = train_test_split(X_train, y_accent_train, test_size=0.15, random_state=10)
#X_val = (X_val - mean)/std
#X_val= np.expand_dims(X_val, axis=3)

print ('Train set:', X_train.shape,  y_accent_train.shape)
print ('Test set:', X_test.shape,  y_accent_test.shape)
#print ('validation set:', X_val.shape,  y_accent_val.shape)

Train set: (1710, 10, 862, 1) (1710, 200)
Test set: (428, 10, 862, 1) (428, 200)


In [34]:
newX_train = X_train.reshape(1710, 8620)
newX_test = X_test.reshape(428, 8620)

y_label_train = y_accent_train.argmax(axis=1)
y_label_test =  y_accent_test.argmax(axis=1)

print ('Train set:', newX_train.shape,  y_label_train.shape)
print ('Test set:', newX_test.shape,  y_label_test.shape)

Train set: (1710, 8620) (1710,)
Test set: (428, 8620) (428,)


In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(newX_train)

X_train = scaler.transform(newX_train)
X_test = scaler.transform(newX_test)

# Training and Predicting for Classification
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()
classifier.fit(newX_train, y_label_train)
y_pred = classifier.predict(newX_test)

# print accuracy
acc =  classifier.score(newX_test, y_label_test)
print(acc) 

0.24065420560747663


# 4 classification

In [12]:
English = data[data['native_language']=='english']
French = data[data['native_language']=='french']
Spanish= data[data['native_language']=='spanish']
Arabic = data[data['native_language']=='arabic']
datanew = English.append(French).append(Spanish).append(Arabic)

parent_list_new = datanew['filename']+ '.mp3'

#features = feature_extractor(parent_list, feature_type='mfcc')

features = np.zeros(shape=(906,10,862))
i=0
SAMPLE_RATE = 22050
dire = 'archive/recordings/recordings/'
for file in parent_list_new[:]:
    f_name = str(dire+file)
    y, sr = librosa.load(f_name, sr=SAMPLE_RATE, duration = 20 )     # duration 10s
    mfcc = librosa.feature.mfcc(y=y, sr=SAMPLE_RATE, n_mfcc =10)
    if len(mfcc[1]) < 862:
        offset = 862 - len(mfcc[1])
        mfcc= np.pad(mfcc,((0,0),(offset,0)), 'constant')
        
    features[i,:,:]=mfcc
    i+=1

D:\py\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


In [13]:
np.save("knn_features_4class.npy",features)
X = np.load("knn_features_4class.npy")
print(np.shape(X))

(906, 10, 862)


In [14]:
#PCA 
from sklearn.decomposition import PCA
proj = np.zeros(shape=(906,10,862))

for i in range(906):
    X1 = X[i,:,:]
    pca = PCA(whiten=True)
    X_pca = pca.fit_transform(X1)
    proj[i,:,:] = pca.inverse_transform(X_pca)

X = proj

In [15]:
y_accent = datanew['native_language']

encoder1 = LabelEncoder()
encoder1.fit(y_accent)
y_accent_ = encoder1.transform(y_accent)
y_accent_ = to_categorical(np.array(y_accent_),dtype='float32')
print(np.shape(y_accent_))  #4 categories.

(906, 4)


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_accent_train, y_accent_test = train_test_split(X, y_accent_, test_size=0.2, random_state=10)

mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)

X_train = (X_train - mean)/std
X_test = (X_test - mean)/std
X_train= np.expand_dims(X_train, axis=3)
X_test= np.expand_dims(X_test, axis=3)

#X_train, X_val, y_accent_train, y_accent_val = train_test_split(X_train, y_accent_train, test_size=0.15, random_state=10)
#X_val = (X_val - mean)/std
#X_val= np.expand_dims(X_val, axis=3)

print ('Train set:', X_train.shape,  y_accent_train.shape)
print ('Test set:', X_test.shape,  y_accent_test.shape)
#print ('validation set:', X_val.shape,  y_accent_val.shape)


Train set: (724, 10, 862, 1) (724, 4)
Test set: (182, 10, 862, 1) (182, 4)


In [27]:
newX_train = X_train.reshape(724, 8620)
newX_test = X_test.reshape(182, 8620)

y_label_train = y_accent_train.argmax(axis=1)
y_label_test =  y_accent_test.argmax(axis=1)

print ('Train set:', newX_train.shape,  y_label_train.shape)
print ('Test set:', newX_test.shape,  y_label_test.shape)

Train set: (724, 8620) (724,)
Test set: (182, 8620) (182,)


In [28]:
# Feature Scaling for Classification
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(newX_train)

newX_train = scaler.transform(newX_train)
newX_test = scaler.transform(newX_test)

# Training and Predicting for Classification
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier()
classifier.fit(newX_train, y_label_train)
y_pred = classifier.predict(newX_test)

# print accuracy
acc =  classifier.score(newX_test, y_label_test)
print(acc) 

0.6208791208791209
